In [1]:
import pandas as pd
import os
import sys
root_dir = os.path.dirname(os.path.dirname(os.getcwd())) ## TODO: find way to put this into some global settings
if root_dir not in sys.path:
    sys.path.append(root_dir)

from references import common_cfg, istat_kpi

In [3]:
## Preprocess school list
statali = pd.read_csv('../../data/raw/cultura/stat_anagrScuole1617.csv').set_index('CODICESCUOLA')
paritarie = pd.read_csv('../../data/raw/cultura/parit_anagrScuole1617.csv').set_index('CODICESCUOLA')
assert set(paritarie.columns) < (set(statali.columns)), 'unexpected columns in paritarie'
assert ~any(statali.SEDESCOLASTICA.isnull()), 'missing info for statali'

# add statale/paritaria flag
statali['bStatale'] = True
paritarie['bStatale'] = False

anagrafica = pd.concat([statali, paritarie], axis=0)
# patch tutte le paritarie come sedi scolastiche
anagrafica.SEDESCOLASTICA.fillna('SI', inplace=True)
anagrafica.to_csv('anagrScuoleAll.csv', sep=';', decimal = ',') 

# drop non-sedi scolastiche
anagrafica = anagrafica[anagrafica.SEDESCOLASTICA=='SI']

FileNotFoundError: File b'../data/cultura/raw/stat_anagrScuole1617.csv' does not exist

In [ ]:
# Filter for each city and export to interim!
listaCitta = common_cfg.listaCitta

for cittaScelta in common_cfg.cityList:
    anagraficaCitta = anagrafica[anagrafica.DESCRIZIONECOMUNE.apply(str.lower) == cittaScelta.lower()]
    anagraficaCitta.to_csv('../../data/interim/%s_anagrafica_scuole.csv' % cittaScelta, sep=';', decimal = ',') 